In [3]:
import numpy as np
import torch
from torch import nn
import tensorflow as tf

# Torch Tensor

Reference:
* https://pytorch.org/docs/stable/tensors.html

In [ ]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

In [ ]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

In [ ]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

In [ ]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

In [ ]:
device = "cuda"
tensor = torch.rand(3,4).to(device)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

In [ ]:
# We move our tensor to the GPU if available
if torch.cuda.is_available():
  tensor = tensor.to('cuda')

In [ ]:
tensor = torch.ones(4, 4)
print('First row: ',tensor[0])
print('First column: ', tensor[:, 0])
print('Last column:', tensor[..., -1])
tensor[:,1] = 0
print(tensor)

In [ ]:
# cat by column with dim=1
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

In [ ]:
# cat by row with dim=0
t1 = torch.cat([tensor, tensor, tensor], dim=0)
print(t1)

In [ ]:
print(f"tensor.shape: {tensor.shape}\n")

In [ ]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(tensor)
torch.matmul(tensor, tensor.T, out=y3)


# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

In [ ]:
agg = tensor.sum()
# for python numerical value
agg_item = agg.item()
print(agg_item, type(agg_item))

In [ ]:
# in-place operation
print(tensor, "\n")
tensor.add_(5)
print(tensor)

In [ ]:
# conversion between numpy and tensor
t = torch.ones(5)
print(f"t: {t}")
n = t.numpy()
print(f"n:{n}")

t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

n = np.zeros(5)
t = torch.from_numpy(n)
np.add(n, 1, out=n)
print(f"t: {t}")
print(f"n: {n}")

# Keras sparse_categorical_crossentropy() vs. Torch nn.CrossEntropyLoss()

Reference:
* https://pytorch.org/docs/1.9.1/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss
* https://www.tensorflow.org/api_docs/python/tf/keras/metrics/sparse_categorical_crossentropy

1. Formula

Binary cross entropy - https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html#cross-entropy:
$$ \mathbb{J}(W) = -\frac{1}{N} \sum_{i=1}^N [y_i \log(\hat{y_i}) + (1-y_i) \log(1 - \hat{y_i})] $$

Categorical cross entropy - https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy#answer-41923:
$$ \mathbb{J}(W) = -\frac{1}{N} \sum_{s \in S} \sum_{c \in C} 1_{s \in c} \log p(s \in C) $$

2. Implementation

Keras' SparseCategoricalCrossentropy:
* https://github.com/keras-team/keras/blob/v2.8.0/keras/losses.py#L1834-L1863

In [23]:
y_true = [1, 2]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]

In [24]:
torch.tensor(y_pred).shape

torch.Size([2, 3])

In [25]:
torch.tensor(y_true, dtype=torch.long).shape

torch.Size([2])

In [47]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)(y_true, y_pred)
np.average(loss.numpy())

1.1769392

In [46]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred)
loss.numpy()

1.1769392

In [33]:
# manual calculation for SparseCategoricalCrossentropy
[-np.log(y_pred[i][y_index]) for i, y_index in enumerate(y_true)]

[0.05129329438755058, 2.3025850929940455]

In [45]:
nn.CrossEntropyLoss(reduction="none", weight=)(torch.tensor(y_pred), torch.tensor(y_true, dtype=torch.long))

tensor([0.5841, 1.3897])

In [38]:
torch.tensor(y_pred)

tensor([[0.0500, 0.9500, 0.0000],
        [0.1000, 0.8000, 0.1000]])

In [41]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)

In [50]:
y_pred

[[0.05, 0.95, 0], [0.1, 0.8, 0.1]]

In [53]:
torch.zeros(torch.tensor(y_pred).shape)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [75]:
from torch.nn.functional import one_hot

def sparse_cross_entropy(y_true, y_pred):
    return torch.mean(-torch.log(torch.sum(one_hot(torch.tensor(y_true), num_classes=len(y_pred[0])) * torch.tensor(y_pred), axis=1)))

In [76]:
sparse_cross_entropy(y_true, y_pred)

tensor(1.1769)

In [44]:
input

tensor([[-0.7163, -0.6112, -1.8985,  0.0598,  0.1774],
        [-1.4910, -0.6921, -2.2275, -0.0864,  0.6479],
        [ 0.5577,  0.1482,  0.7020,  1.2704, -0.0568]], requires_grad=True)

# Torch Autograd

Reference:
* https://towardsdatascience.com/papers-simplified-gradients-without-backpropagation-96e8533943fc

In [4]:
import torch

def function(theta):
    return (theta[0]+theta[1])*theta[2]

theta=torch.tensor([3.,-1.,2.], requires_grad=True)
result=function(theta)

In [5]:
result.backward()
print(result)
print(theta.grad)

tensor(4., grad_fn=<MulBackward0>)
tensor([2., 2., 2.])
